# Fine-tuning DeepSeek-R1 for Medical SOAP Note Generation
- This notebook demonstrates how to fine-tune DeepSeek-R1 (1.5B parameter model) to generate medical SOAP notes from doctor-patient dialogues.
- SOAP notes are structured medical documentation including Subjective, Objective, Assessment, and Plan sections.

# Setup Environment
  First, we'll mount Google Drive, install required packages, and set up our environment.

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
# Install required packages
!pip install -qU transformers==4.48.3 datasets==3.2.0 optimum==1.24.0
!pip install -qU openai==1.61.0 wandb
!pip install -qU json-repair==0.29.1
!pip install -qU faker==35.2.0
#!pip install -qU vllm==0.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Clone LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory && pip install -e .

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 357, done.
remote: Counting objects: 100% (357/357), done.
remote: Compressing objects: 100% (274/274), done.
remote: Total 357 (delta 73), reused 310 (delta 68), pack-reused 0 (from 0)
Receiving objects: 100% (357/357), 9.73 MiB | 15.47 MiB/s, done.
Resolving deltas: 100% (73/73), done.
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 29.5 MB/s eta 0:00:00

# Authentication Setup
We'll set up authentication for Weights & Biases (for experiment tracking) and Hugging Face (for model downloading and uploading).

In [ ]:
# Login to WandB and HuggingFace
from google.colab import userdata
import wandb

wandb.login(key=userdata.get('wandb'))
hf_token = userdata.get('huggingface')
!huggingface-cli login --token {hf_token}

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hazemwalied2003 (hazemwalied2003-cairo-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `notebooks_token` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `notebooks_token`


# Import Libraries
Now we'll import the essential libraries for our fine-tuning process.

In [ ]:
import json
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests

from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime

import json_repair

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Set directories and model
data_dir = "/gdrive/MyDrive/deepseek-finetune-v2"
base_model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

device = "cuda"
torch_dtype = None

# Data Loading and Preparation
We'll load the medical dialogue dataset and format it for fine-tuning.

In [ ]:
# Helper function to parse JSON
def parse_json(text):
    try:
        return json_repair.loads(text)
    except:
        return None

# Load dataset
from datasets import load_dataset, concatenate_datasets

dataset = load_dataset("omi-health/medical-dialogue-to-soap-summary")

# Combine train and test for simpler handling
dataset['train'] = concatenate_datasets([dataset['train'], dataset['test']])
dataset['train'] = concatenate_datasets([dataset['train'], dataset['validation']])
dataset.pop('test')
dataset.pop('validation')

print(f"Total samples in dataset: {len(dataset['train'])}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

train.json:   0%|          | 0.00/154M [00:00<?, ?B/s]

validation.json:   0%|          | 0.00/8.31M [00:00<?, ?B/s]

test.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

Total samples in dataset: 10000


# Define SOAP Summary Task
We'll use Pydantic to create a schema for our SOAP notes, which will help structure the model's output.

In [ ]:
class SOAPSummary(BaseModel):
    subjective: str = Field(...,
                           description="Patient's subjective experiences, symptoms, and concerns as reported in the dialogue.")
    objective: str = Field(...,
                          description="Objective medical findings, vital signs, and examination results mentioned in the dialogue.")
    assessment: str = Field(...,
                           description="Clinician's assessment and diagnosis of the patient's condition based on the dialogue.")
    plan: str = Field(...,
                     description="Treatment plan, medications, follow-up instructions, and recommendations discussed in the dialogue.")

# Prepare Training Data
Now we'll format the dataset for fine-tuning with the LLaMA-Factory framework.

In [ ]:
# Create directory if it doesn't exist
os.makedirs(join(data_dir, "datasets", "llamafactory-finetune-data"), exist_ok=True)

In [ ]:
# Format Dataset for Fine-tuning with JSON structure
llm_finetunning_data = []

system_message = "\n".join([
    "You are a medical professional tasked with creating structured SOAP notes from patient-doctor dialogues.",
    "Generate a comprehensive SOAP summary that includes Subjective, Objective, Assessment, and Plan sections.",
    "Your summary should be concise yet comprehensive, capturing all relevant medical information.",
    "Output must be in valid JSON format according to the provided schema.",
    "Do not include any text outside the JSON structure."
])

# Process dataset
for i, entry in enumerate(tqdm(dataset['train'])):
    dialogue = entry['dialogue']
    soap_summary = entry['soap']

    components = {}

    if "S:" in soap_summary and "O:" in soap_summary:
        s_text = soap_summary.split("S:")[1].split("O:")[0].strip()
        components["subjective"] = s_text

    if "O:" in soap_summary and "A:" in soap_summary:
        o_text = soap_summary.split("O:")[1].split("A:")[0].strip()
        components["objective"] = o_text

    if "A:" in soap_summary and "P:" in soap_summary:
        a_text = soap_summary.split("A:")[1].split("P:")[0].strip()
        components["assessment"] = a_text

    if "P:" in soap_summary:
        p_text = soap_summary.split("P:")[1].strip()
        components["plan"] = p_text

    # If we couldn't parse properly, use placeholders to prevent errors
    if "subjective" not in components:
        components["subjective"] = "Information not available"
    if "objective" not in components:
        components["objective"] = "Information not available"
    if "assessment" not in components:
        components["assessment"] = "Information not available"
    if "plan" not in components:
        components["plan"] = "Information not available"

    # Create JSON output for training
    json_output = json.dumps(components, ensure_ascii=False)

    llm_finetunning_data.append({
        "system": system_message,
        "instruction": "\n".join([
            "# Patient-Doctor Dialogue:",
            dialogue,
            "",
            "# Task:",
            "Generate a structured SOAP summary from the above medical dialogue.",
            "The summary should include Subjective, Objective, Assessment, and Plan sections in JSON format.",
            "",
            "# JSON Schema:",
            json.dumps(SOAPSummary.model_json_schema(), ensure_ascii=False),
            "",
            "# SOAP Summary:",
            "```json"
        ]),
        "input": "",
        "output": json_output + "\n```",
        "history": []
    })

  0%|          | 0/10000 [00:00<?, ?it/s]

# Examine Sample Data
Let's look at a sample from our dataset to understand what we're working with.

In [ ]:
print(dataset['train'][121]['soap'])

S: The patient, with a history of chronic obstructive pulmonary disease (COPD), presents with shortness of breath and a productive cough. He reports worsening dyspnea and a productive cough eight weeks post initial admission.
O: Initial CT angiogram revealed spiculated nodules in bilateral lower lobes and right hilar lymphadenopathy. Pathology confirmed a moderately differentiated nonkeratinizing squamous cell carcinoma of the left lung nodule and highly atypical large lymphocytes in the right lymph node, suggestive of large B-cell lymphoma. Recent CT chest with contrast showed a large, centrally necrotic mass measuring 18.1 cm x 8 cm. Vital signs indicate hemodynamic stability with no increased oxygen requirements, but physical exam shows facial swelling, jugular venous distention, diffuse wheezing, bilateral basal crackles, and bilateral pitting pedal edema.
A: Primary diagnosis of squamous cell carcinoma of the lung with concurrent large B-cell lymphoma. Differential diagnosis inclu

In [ ]:
print(llm_finetunning_data[121])

{'system': 'You are a medical professional tasked with creating structured SOAP notes from patient-doctor dialogues.\nGenerate a comprehensive SOAP summary that includes Subjective, Objective, Assessment, and Plan sections.\nYour summary should be concise yet comprehensive, capturing all relevant medical information.\nOutput must be in valid JSON format according to the provided schema.\nDo not include any text outside the JSON structure.', 'instruction': '# Patient-Doctor Dialogue:\nDoctor: Hello, can you tell me about your past medical history? Have you been diagnosed with any chronic conditions?\nPatient: Yes, I have chronic obstructive pulmonary disease (COPD).\nDoctor: I see. What brings you here today?\nPatient: I\'ve been feeling short of air and have had a productive cough.\nDoctor: Alright, we\'ll need to do a CT angiogram to check for pulmonary embolism. We\'ll also look for any other abnormalities in your lungs.\n(CT Angiogram results)\nDoctor: The CT angiogram showed spicul

# Split Dataset
We'll split our dataset into training and evaluation sets.

In [ ]:
# Shuffle data with a fixed seed for reproducibility
random.Random(101).shuffle(llm_finetunning_data)

# Split into train and evaluation sets (90/10 split)
train_ratio = 0.9
train_size = int(len(llm_finetunning_data) * train_ratio)

train_ds = llm_finetunning_data[:train_size]
eval_ds = llm_finetunning_data[train_size:]

print(f"Training samples: {len(train_ds)}")
print(f"Evaluation samples: {len(eval_ds)}")

Training samples: 9000
Evaluation samples: 1000


In [ ]:
# Save datasets
with open(join(data_dir, "datasets", "llamafactory-finetune-data", "train.json"), "w", encoding="utf8") as dest:
    json.dump(train_ds, dest, ensure_ascii=False)

with open(join(data_dir, "datasets", "llamafactory-finetune-data", "val.json"), "w", encoding="utf8") as dest:
    json.dump(eval_ds, dest, ensure_ascii=False)

print(f"Datasets saved to {join(data_dir, 'datasets', 'llamafactory-finetune-data')}")

Datasets saved to /gdrive/MyDrive/deepseek-finetune-v2/datasets/llamafactory-finetune-data


# Test Base Model
Before fine-tuning, let's test the base model's capabilities on our task.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    torch_dtype=torch_dtype
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
# Test with a sample
sample_dialogue = dataset['train'][121]['dialogue']
print("Sample dialogue:\n", sample_dialogue[:300], "...\n")

# Create system message that emphasizes JSON output
system_message = "\n".join([
    "You are a medical professional tasked with creating structured SOAP notes from patient-doctor dialogues.",
    "Generate a comprehensive SOAP summary that includes Subjective, Objective, Assessment, and Plan sections.",
    "Your summary should be concise yet comprehensive, capturing all relevant medical information.",
    "Output must be in valid JSON format according to the provided schema.",
    "Do not include any text outside the JSON structure."
])

# Create testing prompt with JSON schema
sample_messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": "\n".join([
            "# Patient-Doctor Dialogue:",
            sample_dialogue,
            "",
            "# Task:",
            "Generate a structured SOAP summary from the above medical dialogue.",
            "The summary should include Subjective, Objective, Assessment, and Plan sections in JSON format.",
            "",
            "# JSON Schema:",
            json.dumps(SOAPSummary.model_json_schema(), ensure_ascii=False),
            "",
            "# SOAP Summary:",
            "```json"
        ])
    }
]

# Apply chat template
text = tokenizer.apply_chat_template(
    sample_messages,
    tokenize=False,
    add_generation_prompt=True
)

# Generate response
model_inputs = tokenizer([text], return_tensors="pt", padding=True).to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False,
    temperature=0.1,
    repetition_penalty=1.1
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print("Model response preview:")
print(response)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexp

Sample dialogue:
 Doctor: Hello, can you tell me about your past medical history? Have you been diagnosed with any chronic conditions?
Patient: Yes, I have chronic obstructive pulmonary disease (COPD).
Doctor: I see. What brings you here today?
Patient: I've been feeling short of air and have had a productive cough.
 ...

Model response preview:
Alright, so I'm trying to create a SOAP summary from this medical dialogue between a doctor and a patient. First, I need to understand what each section of SOAP stands for. SOAP stands for Subjective, Objective, Assessment, and Plan. Each of these sections needs to capture different aspects of the conversation.

Starting with the **Subjective** section, this part should reflect the patient's personal experiences and feelings. In the dialogue, the patient mentions having chronic obstructive pulmonary disease (COPD), has shortness of breath, and a productive cough. They also report some physical symptoms like facial swelling and edema. So, my obj

# Validate Model Output
Now we'll check if the model's output conforms to our JSON schema.

In [ ]:
print("\nAttempting to parse JSON response:")
try:
    # Find JSON block if the model wrapped it with ```json and ```
    if "```json" in response and "```" in response.split("```json", 1)[1]:
        json_str = response.split("```json", 1)[1].split("```", 1)[0].strip()
    else:
        # If no code blocks, try to parse the whole response
        json_str = response.strip()

    # Clean up any trailing commas that might cause parsing errors
    json_str = json_str.replace(",\n}", "\n}")
    json_str = json_str.replace(",\n]", "\n]")

    parsed_json = json_repair.loads(json_str)
    print("Successfully parsed JSON:")
    print(json.dumps(parsed_json, indent=2))

    try:
        soap_summary = SOAPSummary(**parsed_json)
        print("\nValidation successful! All required fields are present.")
    except Exception as e:
        print(f"\nValidation error: {e}")
        print("JSON does not match the expected schema.")

except Exception as e:
    print(f"Failed to parse JSON: {e}")
    print("Raw response may not contain valid JSON.")

# Add a function to properly format SOAP for reporting/display
def format_soap_display(soap_dict):
    """Format SOAP dictionary into readable text format"""
    formatted = []
    if "subjective" in soap_dict:
        formatted.append(f"S: {soap_dict['subjective']}")
    if "objective" in soap_dict:
        formatted.append(f"O: {soap_dict['objective']}")
    if "assessment" in soap_dict:
        formatted.append(f"A: {soap_dict['assessment']}")
    if "plan" in soap_dict:
        formatted.append(f"P: {soap_dict['plan']}")
    return "\n\n".join(formatted)

# If we successfully parsed JSON, display it in the traditional SOAP format
if 'parsed_json' in locals():
    print("\nFormatted SOAP Summary:")
    print(format_soap_display(parsed_json))

# Compare with ground truth
print("\nGround truth SOAP summary:")
print(dataset['train'][121]['soap'])


Attempting to parse JSON response:
Successfully parsed JSON:
{
  "subjective": {
    "description": "Patient's subjective experiences, symptoms, and concerns.",
    "title": "Subjective",
    "type": "string"
  },
  "objective": {
    "description": "Objective medical findings, vital signs, and examination results.",
    "title": "Objective",
    "type": "string"
  },
  "assessment": {
    "description": "Clinician's assessment and diagnosis of the patient's condition.",
    "title": "Assessment",
    "type": "string"
  },
  "plan": {
    "description": "Treatment plan, medications, follow-up instructions, and recommendations discussed in the dialogue.",
    "title": "Plan",
    "type": "string"
  }
}

Validation error: 4 validation errors for SOAPSummary
subjective
  Input should be a valid string [type=string_type, input_value={'description': "Patient'...tive', 'type': 'string'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type
object

# Configure LLaMA-Factory
We'll set up LLaMA-Factory to recognize our custom datasets.

In [ ]:
dataset_info = {
    "soap_finetune_train": {
        "file_name": f"{data_dir}/datasets/llamafactory-finetune-data/train.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    },
    "soap_finetune_val": {
        "file_name": f"{data_dir}/datasets/llamafactory-finetune-data/val.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    }
}

# Write the dataset configuration to a JSON file
with open("/content/LLaMA-Factory/data/dataset_info.json", "r") as f:
    existing_info = json.load(f)

# Update with our new entries
existing_info.update(dataset_info)

# Write back the updated configuration
with open("/content/LLaMA-Factory/data/dataset_info.json", "w") as f:
    json.dump(existing_info, f, indent=4)

print("Dataset configuration updated successfully!")

Dataset configuration updated successfully!


# LLaMA Factory Fine-tuning Configuration
We'll create a configuration file for LLaMA-Factory that specifies our fine-tuning settings.

In [ ]:
%%writefile /content/LLaMA-Factory/examples/train_lora/soap_finetune.yaml

### model
model_name_or_path: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 16
lora_alpha: 32      # Standard setting (2x rank)
lora_dropout: 0.05
lora_target: all

### dataset
dataset: soap_finetune_train
eval_dataset: soap_finetune_val
template: deepseek
cutoff_len: 2048
overwrite_cache: true
preprocessing_num_workers: 4

### output
output_dir: /gdrive/MyDrive/deepseek-finetune-v2/models/
logging_steps: 50
save_steps: 200
plot_loss: true

### train
per_device_train_batch_size: 1  # Minimum batch size for T4
gradient_accumulation_steps: 16
learning_rate: 5.0e-5
num_train_epochs: 1.5
lr_scheduler_type: cosine
warmup_ratio: 0.05
fp16: true
gradient_checkpointing: true  # Enable gradient checkpointing to save memory
max_grad_norm: 0.3
ddp_timeout: 180000000
bf16: false

### eval
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 200
max_new_tokens: 1024

### reports
report_to: wandb
run_name: deepseek-finetune-v2

# Hub upload configuration
push_to_hub: true
export_hub_model_id: "hazem74/soap-summary-deepseek-v2"
hub_private_repo: true
hub_strategy: checkpoint

Writing /content/LLaMA-Factory/examples/train_lora/soap_finetune.yaml


# Start Fine-tuning
Now we'll start the fine-tuning process using LLaMA-Factory.

In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/soap_finetune.yaml

2025-05-04 13:03:10.047828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746363790.067834    6792 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746363790.074008    6792 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-04 13:03:10.102369: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO 05-04 13:03:20 __init__.py:190] Automatically detected platform cuda.
[INFO|2025-05-04 13:03:23] llamafactory.hp

# LLaMA Factory finetuning Configuration (continue from checkpoint-600)
If training was interrupted or you want to continue from a checkpoint, you can use this configuration:

In [ ]:
%%writefile /content/LLaMA-Factory/examples/train_lora/soap_finetune.yaml

### model
model_name_or_path: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
trust_remote_code: true

### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 16
lora_alpha: 32      # Standard setting (2x rank)
lora_dropout: 0.05
lora_target: all

### checkpoint to resume from
resume_from_checkpoint: /gdrive/MyDrive/deepseek-finetune-v2/models/checkpoint-600

### dataset
dataset: soap_finetune_train
eval_dataset: soap_finetune_val
template: deepseek
cutoff_len: 2048
overwrite_cache: true
preprocessing_num_workers: 4

### output
output_dir: /gdrive/MyDrive/deepseek-finetune-v2/models/
logging_steps: 50
save_steps: 200
plot_loss: true

### train
per_device_train_batch_size: 1  # Minimum batch size for T4
gradient_accumulation_steps: 16
learning_rate: 5.0e-5
num_train_epochs: 1.5
lr_scheduler_type: cosine
warmup_ratio: 0.05
fp16: true
gradient_checkpointing: true  # Enable gradient checkpointing to save memory
max_grad_norm: 0.3
ddp_timeout: 180000000
bf16: false

### eval
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 200
max_new_tokens: 1024

### reports
report_to: wandb
run_name: deepseek-finetune-v2

# Hub upload configuration
push_to_hub: true
export_hub_model_id: "hazem74/soap-summary-deepseek-v2"
hub_private_repo: true
hub_strategy: checkpoint

Writing /content/LLaMA-Factory/examples/train_lora/soap_finetune.yaml


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/soap_finetune.yaml

2025-05-06 09:30:07.206352: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746523807.475276    4763 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746523807.545478    4763 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-06 09:30:08.117477: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO 05-06 09:30:18 __init__.py:190] Automatically detected platform cuda.
[INFO|2025-05-06 09:30:21] llamafactory.hp

# Testing the Fine-tuned Model
Now let's test our fine-tuned model on some examples.

In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.7 MB/s eta 0:00:00


In [ ]:
import os
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Set up offload directory
offload_dir = "/tmp/offload_dir"
os.makedirs(offload_dir, exist_ok=True)

# Manually specify the checkpoint path
adapter_path = "/gdrive/MyDrive/deepseek-finetune-v2/models/checkpoint-843"
print(f"Using adapter from checkpoint: {adapter_path}")

# Base model ID
base_model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
print(f"Using base model: {base_model_id}")

# Setup quantization config to reduce memory usage
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# Load base model with quantization
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    offload_folder=offload_dir,
    torch_dtype=torch.float16
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# Load adapter weights
model = PeftModel.from_pretrained(model, adapter_path)

print("Model loaded successfully!")

Using adapter from checkpoint: /gdrive/MyDrive/deepseek-finetune-v2/models/checkpoint-843
Using base model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Model loaded successfully!


# Create Helper Function for SOAP Generation
We'll create a utility function to generate SOAP notes with our fine-tuned model.

In [ ]:
def generate_soap_summary(dialogue, max_tokens=2048):
    messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": "\n".join([
            "# Patient-Doctor Dialogue:",
            dialogue,
            "",
            "# Task:",
            "Generate a structured SOAP summary from the above medical dialogue.",
            "The summary should include Subjective, Objective, Assessment, and Plan sections in JSON format.",
            "",
            "# JSON Schema:",
            json.dumps(SOAPSummary.model_json_schema(), ensure_ascii=False),
            "",
            "# SOAP Summary:",
            "```json"
        ])
    }
]

    # Apply prompt formatting
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt", padding=True).to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=max_tokens,
            do_sample=False
        )

    # Trim prompt from output
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]



dialogue = dataset['train'][508]['dialogue']
# Print a short preview of the dialogue
print(f"Dialogue preview: {dialogue}...\n")

# Generate and print the summary
summary = generate_soap_summary(dialogue)
print("Generated SOAP Summary:")
print(summary)
print("-" * 80)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexp

Dialogue preview: Doctor: Hello, can you please tell me your medical history and any medications you are currently taking?
Patient: Hi, I'm a 50-year-old woman with a history of HIV and I've been non-compliant with my highly active antiretroviral therapy (HAART).
Doctor: Oh, I see. What brings you here today?
Patient: I've been experiencing confusion, fevers, fatigue, and night sweats for about a month now.
Doctor: Thank you for sharing that. We've conducted some laboratory tests during your admission. The findings show that your hemoglobin is at 4.6 g/dL, lactate at 2.8, and INR at 3.9. Your liver function tests (LFTs) are normal. Your CD4 count is low at 51/mm3, but your HIV viral load is less than 20 and your leukocyte count is normal.
Patient: Oh, okay. So, what does that mean?
Doctor: Well, we've given you blood transfusions to help with your low hemoglobin and hematocrit levels, but we haven't been able to identify a source of bleeding. Meanwhile, you've been febrile and hypotens

# Cost Estimation for Inference
Let's estimate the computational cost of running inference with our model.

In [ ]:
from tqdm.auto import tqdm
import random
from faker import Faker
from datetime import datetime
import torch
import warnings; warnings.filterwarnings("ignore", message="The attention mask and the pad token id were not set.*")

# Initialize Faker
fake = Faker()
start_time = datetime.now()

# Start tracking tokens
input_tokens = 0
output_tokens = 0

num_samples = 30

for i in tqdm(range(num_samples)):
    # Generate random text using Faker
    dialogue = fake.text(max_nb_chars=random.randint(400, 800))

    # Create the messages
    messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": "\n".join([
            "# Patient-Doctor Dialogue:",
            dialogue,
            "",
            "# Task:",
            "Generate a structured SOAP summary from the above medical dialogue.",
            "The summary should include Subjective, Objective, Assessment, and Plan sections in JSON format.",
            "",
            "# JSON Schema:",
            json.dumps(SOAPSummary.model_json_schema(), ensure_ascii=False),
            "",
            "# SOAP Summary:",
            "```json"
        ])
    }
]

    # Get the input token count
    input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    current_input_tokens = len(tokenizer.encode(input_text))
    input_tokens += current_input_tokens

    # Generate the response
    response = generate_soap_summary(dialogue)

    # Get the output token count
    current_output_tokens = len(tokenizer.encode(response))
    output_tokens += current_output_tokens

    if i < 3:  # Just show stats for first few samples
        print(f"\nSample {i+1}:")
        print(f"  Random text length (chars): {len(dialogue)}")
        print(f"  Input tokens: {current_input_tokens}")
        print(f"  Output tokens: {current_output_tokens}")
        print(f"  Ratio (output/input): {current_output_tokens/current_input_tokens:.2f}")

# Calculate totals
total_time = (datetime.now() - start_time).total_seconds()
total_tokens = input_tokens + output_tokens

print(f"\n--- Cost Estimation Summary ---")
print(f"Total Processing Time: {total_time:.2f} seconds")
print(f"Average Time Per Sample: {total_time/num_samples:.2f} seconds")
print(f"Input Tokens: {input_tokens} (Average: {input_tokens/num_samples:.1f} per sample)")
print(f"Output Tokens: {output_tokens} (Average: {output_tokens/num_samples:.1f} per sample)")
print(f"Total Tokens: {total_tokens} (Average: {total_tokens/num_samples:.1f} per sample)")
print(f"Output/Input Token Ratio: {output_tokens/input_tokens:.2f}")

# Estimated cost calculation
input_cost_per_1k = 0.0015  # Example rate: $0.0015 per 1K input tokens
output_cost_per_1k = 0.002  # Example rate: $0.002 per 1K output tokens

estimated_input_cost = (input_tokens / 1000) * input_cost_per_1k
estimated_output_cost = (output_tokens / 1000) * output_cost_per_1k
total_estimated_cost = estimated_input_cost + estimated_output_cost

print(f"\n--- Estimated Costs (using example rates) ---")
print(f"Input Cost: ${estimated_input_cost:.4f}")
print(f"Output Cost: ${estimated_output_cost:.4f}")
print(f"Total Estimated Cost: ${total_estimated_cost:.4f}")
print(f"Cost Per Sample: ${total_estimated_cost/num_samples:.4f}")

  0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attentio


Sample 1:
  Random text length (chars): 674
  Input tokens: 417
  Output tokens: 727
  Ratio (output/input): 1.74


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



Sample 2:
  Random text length (chars): 371
  Input tokens: 367
  Output tokens: 581
  Ratio (output/input): 1.58


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



Sample 3:
  Random text length (chars): 733
  Input tokens: 422
  Output tokens: 578
  Ratio (output/input): 1.37


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end gene


--- Cost Estimation Summary ---
Total Processing Time: 2162.28 seconds
Average Time Per Sample: 72.08 seconds
Input Tokens: 12005 (Average: 400.2 per sample)
Output Tokens: 19951 (Average: 665.0 per sample)
Total Tokens: 31956 (Average: 1065.2 per sample)
Output/Input Token Ratio: 1.66

--- Estimated Costs (using example rates) ---
Input Cost: $0.0180
Output Cost: $0.0399
Total Estimated Cost: $0.0579
Cost Per Sample: $0.0019


In [ ]:
print("No. of Tokens per second = "+str(total_tokens/total_time))

No. of Tokens per second = 14.778838258084043


# Pushing Model to **HuggingFace** Hub
Now we'll push our fine-tuned model to the Hugging Face Hub for easy sharing and deployment.

In [ ]:
!pip install -q peft huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.9 MB/s eta 0:00:00


In [ ]:
# Log in to Hugging Face
from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('huggingface')
login(token=hf_token)

In [ ]:
# Load the base model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define model paths
base_model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
print(f"Loading base model: {base_model_id}")

# Load the model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto"
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading base model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B


In [ ]:
# Load the adapter
from peft import PeftModel

# Path to your fine-tuned adapter
adapter_path = "/gdrive/MyDrive/deepseek-finetune-v2/models/checkpoint-843"
print(f"Loading adapter from: {adapter_path}")

# Load the adapter onto the base model
model = PeftModel.from_pretrained(base_model, adapter_path)

Loading adapter from: /gdrive/MyDrive/deepseek-finetune-v2/models/checkpoint-843


In [ ]:
# Merge the adapter weights with the base model
print("Merging adapter with base model...")
merged_model = model.merge_and_unload()

Merging adapter with base model...


In [ ]:
# Create a model card
import os

# Your Hugging Face username and model name
your_username = "hazem74"
model_name = "deepseek-soap-summary-v2"
hf_model_repo = f"{your_username}/{model_name}"

# Create a model card
model_card = f"""
---
language:
- en
tags:
- medical
- healthcare
- SOAP notes
- clinical documentation
license: mit
datasets:
- omi-health/medical-dialogue-to-soap-summary
---

# DeepSeek SOAP Summary Generator

This model is fine-tuned to generate SOAP (Subjective, Objective, Assessment, Plan) summaries from patient-doctor dialogues.

## Usage

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("{hf_model_repo}", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("{hf_model_repo}")

# Sample dialogue
dialogue = \"\"\"
Doctor: Hello, how are you feeling today?
Patient: I've been having some chest pain for the last two days.
Doctor: Can you describe the pain?
Patient: It's a sharp pain, mostly on the left side.
\"\"\"

# Format the prompt
system_message = "You are a medical professional tasked with creating SOAP notes from patient-doctor dialogues."
user_content = f\"\"\"
# Patient-Doctor Dialogue:
{{dialogue}}

# Task:
Generate a SOAP summary from the above medical dialogue.
The summary should include Subjective, Objective, Assessment, and Plan sections.

# SOAP Summary:
\"\"\"

messages = [
    {{"role": "system", "content": system_message}},
    {{"role": "user", "content": user_content}}
]

# Generate SOAP summary
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    inputs.input_ids,
    max_new_tokens=512,
    temperature=0.3,
    top_p=0.9
)

soap_summary = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
print(soap_summary)```

Limitations
This model assists healthcare professionals but should not replace human judgment. Always review generated summaries for accuracy.
"""

In [ ]:
# Create a temporary directory and save the model
temp_dir = "/tmp/merged_model"
os.makedirs(temp_dir, exist_ok=True)

print(f"Saving merged model to {temp_dir}")
merged_model.save_pretrained(temp_dir, safe_serialization=True)
tokenizer.save_pretrained(temp_dir)

# Save the model card
with open(os.path.join(temp_dir, "README.md"), "w") as f:
    f.write(model_card)

Saving merged model to /tmp/merged_model


In [ ]:
# Push the model to Hugging Face Hub
from huggingface_hub import HfApi

api = HfApi()
print(f"Pushing model to Hugging Face Hub: {hf_model_repo}")

# Create the repository if it doesn't exist
try:
    api.create_repo(repo_id=hf_model_repo, private=True, exist_ok=True)
    print(f"Repository {hf_model_repo} is ready")
except Exception as e:
    print(f"Note: {e}")

# Upload all files
print("Uploading files...")
api.upload_folder(
    folder_path=temp_dir,
    repo_id=hf_model_repo,
    token=hf_token
)

Pushing model to Hugging Face Hub: hazem74/deepseek-soap-summary-v2
Repository hazem74/deepseek-soap-summary-v2 is ready
Uploading files...


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hazem74/deepseek-soap-summary-v2/commit/1964ace4f10269ed08c9b6c18ce6122e5c9bbc7b', commit_message='Upload folder using huggingface_hub', commit_description='', oid='1964ace4f10269ed08c9b6c18ce6122e5c9bbc7b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hazem74/deepseek-soap-summary-v2', endpoint='https://huggingface.co', repo_type='model', repo_id='hazem74/deepseek-soap-summary-v2'), pr_revision=None, pr_num=None)

# Test the Model from HuggingFace
Now we will test the Model after uploading it to HuggingFace hub

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load once and reuse
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(hf_model_repo)
model = AutoModelForCausalLM.from_pretrained(
    hf_model_repo,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True
)

# System message (keep it same as before)
system_message = "\n".join([
    "You are a medical professional tasked with creating structured SOAP notes from patient-doctor dialogues.",
    "Generate a comprehensive SOAP summary that includes Subjective, Objective, Assessment, and Plan sections.",
    "Your summary should be concise yet comprehensive, capturing all relevant medical information.",
    "Output must be in valid JSON format according to the provided schema.",
    "Do not include any text outside the JSON structure."
])

tokenizer_config.json:   0%|          | 0.00/6.76k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/485 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
# Test dialogue
test_dialogue = """
Doctor: Good morning. What brings you in today?
Patient: Hi, Doctor. I’ve been having a persistent cough for about two weeks now. It gets worse at night, and sometimes I feel short of breath.
Doctor: I see. Any fever, chills, or chest pain?
Patient: No fever or chills, but I do get some mild chest discomfort when I cough a lot.
Doctor: Do you have a history of asthma or any allergies?
Patient: I have mild asthma, but it’s usually well-controlled. I use an inhaler only occasionally.
Doctor: Any recent exposure to someone who was sick?
Patient: Yes, my daughter had a cold last week.
Doctor: Have you noticed any wheezing or phlegm?
Patient: Yes, there’s some wheezing, especially at night, and I’ve been coughing up clear phlegm.
Doctor: How’s your appetite and energy level?
Patient: Appetite is okay, but I’ve been feeling a bit tired lately.
Doctor: Alright. I’ll listen to your lungs now... (examines patient) You have some mild wheezing in both lungs, more on the right side.
Patient: Is it serious?
Doctor: It sounds like an asthma flare-up possibly triggered by a viral infection. Your oxygen levels are normal, and there’s no sign of pneumonia.
Patient: Okay, what should I do?
Doctor: I’ll prescribe a short course of oral steroids and advise using your inhaler regularly for the next few days. Drink plenty of fluids and rest. If your breathing worsens, go to the ER immediately."""

sample_dialogue = dataset['train'][121]['dialogue']

summary = generate_soap_summary(test_dialogue)
print(summary)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Alright, I need to create a structured SOAP summary based on the provided medical dialogue. The user has provided a detailed conversation between a patient and a doctor, which includes the patient's symptoms, history, and treatment plan. My task is to summarize this information in a JSON format with specific sections: Subjective, Objective, Assessment, and Plan.

First, I'll start by identifying the Subjective section. The patient reports a persistent cough lasting about two weeks, worsening at night and sometimes accompanied by shortness of breath. The patient also mentions mild chest discomfort when coughing significantly. There's no fever, chills, or chest pain, but the patient has mild asthma, controlled, and uses an inhaler occasionally. Additionally, the patient experiences wheezing, especially at night, and coughs up clear phlegm. The patient also reports feeling tired and having a history of a cold from a previous illness.

Next, the Objective section should include vital signs